# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from pprint import pprint


In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [23]:
products_by_client = data.groupby(['CustomerID', 'ProductID']).sum()

#Remove 'SalesID'
products_by_client = products_by_client.drop(columns=['SalesID'])
products_by_client.head()

,CustomerID,ProductID,Quantity
0,33,23,1
1,33,51,1
2,33,53,1
3,33,67,1
4,33,70,1


In [25]:
#You will need to group by CustomerID and ProductName and then sum the Quantity field.


df=data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot_table = pd.pivot_table(data, values = 'Quantity', index = 'ProductName', columns = 'CustomerID')
pivot_table = pivot_table.fillna(0)
pivot_table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
pivot_table_transpose = pivot_table.transpose()
customer_similarity_matrix = pd.DataFrame(squareform(pdist(pivot_table_transpose, 'euclidean')),
                                          columns = pivot_table_transpose.index, index = pivot_table_transpose.index)

customer_similarity_matrix.head()


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,10.723805,9.643651,9.899495,10.440307,10.148892,10.723805,10.246951,10.344080,10.148892,...,206.963765,194.766527,199.145675,183.463893,202.259734,198.075743,203.921554,197.949489,218.824130,203.798921
200,10.723805,0.000000,10.677078,10.723805,11.489125,11.045361,10.770330,10.954451,10.677078,10.488088,...,206.269242,194.285872,198.423789,182.543146,202.044550,197.729613,202.723950,197.349943,218.625250,203.953426
264,9.643651,10.677078,0.000000,10.344080,10.198039,10.488088,10.677078,10.583005,10.488088,10.488088,...,206.366179,194.517351,199.153207,182.789496,201.772149,196.817174,203.315026,197.324606,218.029814,203.806281
356,9.899495,10.723805,10.344080,0.000000,10.630146,10.535654,10.630146,9.949874,10.630146,10.440307,...,206.726873,194.643263,199.025124,182.923481,202.264678,197.575302,204.293906,197.322072,218.142155,203.681123
412,10.440307,11.489125,10.198039,10.630146,0.000000,10.677078,10.677078,10.198039,10.770330,10.392305,...,206.985507,193.889144,199.168271,182.805908,202.034650,197.339808,204.066166,197.466453,217.928887,202.960587


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
list = list(customer_similarity_matrix[33].sort_values(ascending=False).index)[:5]

In [8]:
list

[96560, 91443, 97324, 98185, 91777]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
records = df[df['CustomerID'].isin(list)]
records.head()

,CustomerID,ProductName,Quantity
59268,91443,Appetizer - Sausage Rolls,24
59269,91443,Baking Powder,24
59270,91443,Banana Turning,24
59271,91443,Bananas,24
59272,91443,Bandage - Flexible Neon,24


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
records_agg.head()

,,Quantity
CustomerID,ProductName,
91443,Scallops - 10/20,72
97324,Cream Of Tartar,50
98185,Scallops - Live In Shell,50
96560,Chicken - Soup Base,50
97324,Tomato - Tricolor Cherry,50


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

#### Merge the ranked products data frame with the customer product matrix on the ProductName field.

In [11]:
products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
products_similar_customers.head()

,Quantity
ProductName,
Cream Of Tartar,100
"Pasta - Penne, Rigate, Dry",99
Durian Fruit,99
Shrimp - 31/40,99
Bandage - Flexible Neon,98


#### Filter for records where the chosen customer has not purchased the product.


In [12]:
customer33_never_purchased = pivot_table[pivot_table[33] == 0]

In [13]:
customer33_never_purchased.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,25.0


In [27]:
customer33_never_purchased.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,25.0


#### Show the top 5 results.

In [28]:

recomendation= list(customer33_never_purchased.merge(products_similar_customers, on = 'ProductName').sort_values(
    'Quantity',ascending = False).index[0:5])


TypeError: 'list' object is not callable

In [29]:
pprint(recomendation)

NameError: name 'recomendation' is not defined

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

#### Create an empty dictionary that will hold the recommendations for all customers.

In [ ]:
recommendations = {}


#### Create a list of unique CustomerIDs to iterate over

In [ ]:
customers_ids = [data['CustomerID'].unique()]

#### Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created

In [ ]:
for customers in customers_ids:
    for customer in customers:
        lista = list(customer_similarity_matrix[customer].sort_values(ascending=False).index)[:5]
        records = quantity[quantity['CustomerID'].isin(lista)]
        records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
        'Quantity', ascending=False)
        products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
        'Quantity', ascending=False)
        customer_never_purchased = pivot_table[pivot_table[customer] == 0]
        recomendacion = list(customer_never_purchased.merge(
            products_similar_customers, on = 'ProductName').sort_values('Quantity',ascending = False).index[0:5])
        recommendations.update({customer: recomendacion})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
# Hago df desde diccionario y reinicio index:
results = pd.DataFrame.from_dict(recommendations, columns=['Product1', 'Product2', 'Product3', 'Product4',
                                                           'Product5'], orient='index').reset_index(drop=True)
# Cambio el nombre del index:
results.index.names = ['CustomerID']

results.head()


In [ ]:
results = pd.DataFrame.from_dict(recommendations_for_everyone, orient='index').reset_index()
results.columns = ['CustomerID','Prod_01', 'Prod_02', 'Prod_03', 'Prod_04', 'Prod_05']
results.head()

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.